## Import used dependencies

In [1]:
import pandas as pd
import os

## Load the problems as well as the test cases

In [2]:
# Read the problems file
problems = pd.read_json('code4bench_no_loops.json')
test_cases = pd.read_json('code4bench_testcases.json')

## Look at what the test cases and problems look like

In [3]:
print(problems)
print(test_cases)

       id                                         sourceCode fullname name  \
0       2  #include<stdio.h>\r\nint main()\r\n{\r\n    in...     581A    A   
30     66  #include <stdio.h>\r\r\nint main()\r\r\n{\r\r\...     591A    A   
38     69  #include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...     592B    B   
58    102  #include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...     599A    A   
73    159  #include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...     611A    A   
86    194  #include<stdio.h>\r\nint main()\r\n{\r\n    in...     617A    A   
104   230  #include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...     624A    A   
124   262  #include<stdio.h>\r\nint main()\r\n{\r\n\tint ...     630A    A   
139   317  #include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...     638A    A   
146   350  #include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...     645B    B   
161   419  #include<stdio.h>\r\n#include<math.h>\r\nint m...     659A    A   
186   476    main(n){scanf("%d",&n);printf("%d",(n*2+1)/3);}    

## Functions for the test generation

In [65]:
# Read the test template file and print it
path = os.path.join(os.getcwd(), "..","python_modules","testing","test_function.c")

with open(path, 'r') as f:
    template_test_file = f.read()

# Print the content
print(template_test_file)

#include <stdio.h>

// The function declaration
// ---FUNCTION DECLARATION---

// A structure for the test cases
// ---STRUCTURE FOR TEST CASES---

// Initialize test cases
// ---TEST CASES---

// Get the number of test cases
int num_tests = sizeof(tests) / sizeof(tests[0]);

// The main function
int main()
{
    // Keep track of the amount of passed tests
    int passed = 0;

    // For each test case try the function
    for (int i = 0; i < num_tests; i++)
    {
        // Create variables to store the output
        int out1, out2;

        // Run the function
        calculateHipsterSockDays(tests[i].a, tests[i].b, &out1, &out2);

        // Check if the result is correct
        if (out1 == tests[i].out_1 && out2 == tests[i].out_2)
        {
            passed++;
        }
    }
    printf("\nPassed %d out of %d tests.\n", passed, num_tests);
    return 0;
}


# Create a folder for each problem with the context inside

In [35]:
# Create a folder for the problems
# For each of the problems make a folder with the index
for i, problem in problems.iterrows():
    # Get the context of the problem
    context = problem['context']
    
    # Get the path, start with the current directory and add the folder name
    path = os.path.join(os.getcwd(), "../no_loop_problems", f"{i}")

    # Create the folder if it does not exist
    if not os.path.exists(path):
        os.makedirs(path)
    
    # Write the context to a file
    with open(os.path.join(path, 'context.txt'), 'w', encoding="utf-8") as f:
        f.write(context)

   inputdata expectedresult
0        3 1            1 1
1        2 3            2 0
2        7 3            3 2
3    100 100          100 0
4       4 10            4 3
5       6 10            6 2
6       6 11            6 2
7      10 40          10 15
8      11 56          11 22
9      34 30           30 2
10     33 33           33 0
11    100 45          45 27
12    100 23          23 38
13     45 12          12 16
14       1 1            1 0
15     1 100           1 49
16     100 1           1 49
17     68 59           59 4
18     45 99          45 27
19    99 100           99 0
20    100 98           98 1
21     59 12          12 23
22      86 4           4 41
23     68 21          21 23
24    100 11          11 44
25    100 10          10 45
26     15 45          15 15
27     11 32          11 10
28     34 96          34 31
29     89 89           89 0


In [92]:
# Function that replaces the placeholders in the template file
def replace_placeholders(template_file, function_signature, test_cases):
    # Replace the function signature
    template_file = template_file.replace(
        "// ---FUNCTION DECLARATION---", function_signature)
    
    # Replace the struct definition for the test cases
    struct_definition = "typedef struct {\n"

    # Get the part between parantheses in the function signature
    function_signature = function_signature.split("(")[1].split(")")[0]

    # For each parameter in the function signature, add a line to the struct definition
    for i in function_signature.replace("*", "").split(","):
        struct_definition += f"    {i};\n"
    struct_definition += "} TestCase;\n"

    # Replace the struct definition
    template_file = template_file.replace(
        "// ---STRUCTURE FOR TEST CASES---", struct_definition)

    # For each test case, add a line to the test file
    test_cases_lines = "TestCase tests[] = {\n"
    for i, test_case in test_cases.iterrows():
        # Replace the input and output data spaces with commas
        test_case['inputdata'] = test_case['inputdata'].replace(" ", ", ").replace("\n", ", ").replace("\r", " ")
        test_case['expectedresult'] = test_case['expectedresult'].replace(" ", ", ").replace("\n", ", ").replace("\r", " ")

        # Add the input and output to the test file
        test_cases_lines += f"    {{ {test_case['inputdata']}, {test_case['expectedresult']} }},\n"
    test_cases_lines += "};\n"

    # Replace the test cases
    template_file = template_file.replace(
        "// ---TEST CASES---", test_cases_lines)

    return template_file

# Create a folder for the problems
# For each of the problems make a folder with the index
for i, problem in problems.iterrows():
    if i != 0 and i not in [227, 248, 278, 307, 385, 398, 433, 480, 486, 531, 605, 628, 674, 857, 871, 916]:
        # Get the path, start with the current directory and add the folder name
        path = os.path.join(os.getcwd(), "../no_loop_problems", f"{i}")
            
        # Get the test cases
        test_cases_problem = test_cases[test_cases['fullname'] == problem['fullname']]

        # remove the "id" and "fullname" columns
        test_cases_problem = test_cases_problem.drop(columns=['id', 'fullname'])

        # Get the function name and signature in the path
        with open(path + "\specification.h") as f:
            # Take the last line of the file
            function_signature = f.readlines()[-1]

        # Replace the placeholders in the template file
        test_file = replace_placeholders(template_test_file, function_signature, test_cases_problem)

        # Write the test file
        with open(path + "/tests.c", 'w') as f:
            f.write(test_file)

        

100
50
50


In [5]:
index = 24
print(problems.iloc[index])
problems.iloc[index]['sourceCode'].replace("\r", " ").replace("\n", " ").replace("\t", " ")

id                                                          890
sourceCode    #include <stdio.h>\r\r\n#include <stdlib.h>\r\...
fullname                                                   746C
name                                                          C
countline                                                    97
context       C. Tram time limit per test 1 second memory li...
Name: 385, dtype: object


'#include <stdio.h>   #include <stdlib.h>   #include <math.h>      int main()   {   int n,start,end,myt,tramt,tramnow,d;   scanf("%d %d %d",&n,&start,&end);   scanf("%d %d",&tramt,&myt);   scanf("%d %d",&tramnow,&d);   int mytime= abs(start-end)*myt;   int tramtime=0;   if((start>=tramnow&&d==1)||(start<=tramnow&&d==-1))   {       tramtime+=abs(start-tramnow)*tramt;   }   else   {       if(d==-1)       {           tramtime+=(tramnow+start)*tramt;           d=1;       }       else       {           d=-1;           tramtime+=(n-tramnow+n-start)*tramt;          }   }   if((end>=start&&d==1)||(end<=start&&d==-1))       tramtime+=abs(start-end)*tramt;       else       {           if(d==1)           {               tramtime+=(n-start+n-end)*tramt;           }           else           {               tramtime+=(start+end)*tramt;           }       }   if(mytime<tramtime)       printf("%d",mytime);   else       printf("%d",tramtime);       return 0;   }'

In [9]:
problems

,id,sourceCode,fullname,name,countline,context
0,2,#include<stdio.h>\r\nint main()\r\n{\r\n in...,581A,A,15,A. Vasya the Hipster time limit per test 1 sec...
30,66,#include <stdio.h>\r\r\nint main()\r\r\n{\r\r\...,591A,A,19,A. Wizards' Duel time limit per test 2 seconds...
38,69,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,592B,B,15,B. The Monster and the Squirrel time limit per...
58,102,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,599A,A,51,A. Patrick and Shopping time limit per test 1 ...
73,159,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,611A,A,23,A. New Year and Days time limit per test 2 sec...
86,194,#include<stdio.h>\r\nint main()\r\n{\r\n in...,617A,A,11,A. Elephant time limit per test 1 second memor...
104,230,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,624A,A,17,A. Save Luke time limit per test 1 second memo...
124,262,#include<stdio.h>\r\nint main()\r\n{\r\n\tint ...,630A,A,8,A. Again Twenty Five! time limit per test 0.5 ...
139,317,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,638A,A,21,A. Home Numbers time limit per test 1 second m...
146,350,#include<stdio.h>\r\r\nint main()\r\r\n{\r\r\n...,645B,B,33,B. Mischievous Mess Makers time limit per test...


In [91]:
test_cases[test_cases['fullname'] == "591A"]

,id,inputdata,expectedresult,fullname
30,53133,100\r\r\n50\r\r\n50,50,591A
31,53134,199\r\r\n60\r\r\n40,119.4,591A
32,53135,1\r\r\n1\r\r\n1,0.5,591A
33,53138,1\r\r\n500\r\r\n500,0.5,591A
34,53139,1000\r\r\n1\r\r\n1,500,591A
35,53142,1000\r\r\n500\r\r\n500,500,591A
36,53144,987\r\r\n1\r\r\n3,246.75,591A
37,53159,600\r\r\n221\r\r\n279,265.2,591A
